In [0]:
%pip install --upgrade --force-reinstall /dbfs/tmp/insurance-0.0.1-py3-none-any.whl

In [0]:
%restart_python

In [0]:
import os
import time

import mlflow
import pandas as pd
import requests
from databricks import feature_engineering
from pyspark.dbutils import DBUtils
from pyspark.sql import SparkSession

from insurance.config import ProjectConfig
from insurance.serving.feature_serving import FeatureServing

In [0]:
config = ProjectConfig.from_yaml(config_path="../project_config.yml")

# COMMAND ----------
spark = SparkSession.builder.getOrCreate()
dbutils = DBUtils(spark)

fe = feature_engineering.FeatureEngineeringClient()
mlflow.set_registry_uri("databricks-uc")

In [0]:
# get environment variables
os.environ["DBR_TOKEN"] = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
os.environ["DBR_HOST"] = spark.conf.get("spark.databricks.workspaceUrl")

In [0]:
catalog_name = config.catalog_name
schema_name = config.schema_name
feature_table_name = f"{catalog_name}.{schema_name}.house_prices_preds"
feature_spec_name = f"{catalog_name}.{schema_name}.return_predictions"
endpoint_name = "insurance-feature-serving"


In [0]:

train_set = spark.table(f"{catalog_name}.{schema_name}.train_set").toPandas()
test_set = spark.table(f"{catalog_name}.{schema_name}.test_set").toPandas()
df = pd.concat([train_set, test_set])

In [0]:

model = mlflow.sklearn.load_model(f"models:/{catalog_name}.{schema_name}.insurance_model_basic@latest-model")

In [0]:
preds_df = df[["Id", "age", "sex"]]
preds_df["Predicted_SalePrice"] = model.predict(df[config.cat_features + config.num_features])
preds_df = spark.createDataFrame(preds_df)